In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import torch
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd

In [85]:
from src.vit_data import load_cd_data

vit_data = load_cd_data()

In [159]:
from src.data_loader import ViTDataLoader

dataloader = ViTDataLoader(vit_data, split_type='time', split_arg=120.0)


In [163]:
len(dataloader.trainloader.dataset), len(dataloader.validationloader.dataset), len(dataloader.testloader.dataset)

(19921, 2214, 4427)

In [87]:
from src.vit_train import ViTTrainer, load_model_dir, load_model_config
import importlib

config = importlib.import_module(f"config.simple")
vit = load_model_config(config)
trainer = ViTTrainer(vit, dataloader)


In [88]:
device = trainer.device
vit = trainer.vit


In [89]:
trainer.setup()

Creating directory: output/vit_1_20220523_3b54...


In [ ]:
trainer.train()

In [151]:
vit.load_state_dict(torch.load('/Users/trung/Downloads/model.torch.14000', map_location=torch.device('cpu')))

<All keys matched successfully>

In [152]:
trainer.compute_predictions_losses()

In [153]:
print(trainer.perf_str)

Loss:
  Train:	0.958
  Valid:	0.956
  Test: 	0.488

R2:
  Train:	0.128
  Valid:	0.104
  Test: 	-0.491


In [158]:
#trainer.dataloader.trainloader.dataset[0:10]